In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
import pathlib
layers = tf.keras.layers
import os
import cv2
from IPython.display import clear_output

In [2]:
dir_with_images = "D:\\img_align_celeba"
dir_with_labels = "D:\\Labels\\"

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder,self).__init__()
        self.model = None
         
    def call(self,input_feature):
        return self.model.call(input_feature)
    
    def summary(self):
        return self.model.summary()

In [4]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder,self).__init__()
        self.model = None
        
    def call(self,input_feature):
        return self.model.call(input_feature)
    
    def summary(self):
        return self.model.summary()

In [5]:
class Autoncoder(tf.keras.Model):
    def __init__(self, latent_dim = 128):
        super(Autoncoder,self).__init__();
       
        self.encoder = Encoder()
        self.decoder = Decoder()
        
    @tf.function
    def sample(self, eps):
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        return self.encoder.model(x)

    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder.model(z)
        return logits
   
    def summaryEncoder(self):
        return self.encoder.summary()
    
    def summarDecoder(self):
        return self.decoder.summary()
        
    def load(self):
        self.encoder.model = tf.keras.models.load_model('encoder_cae_relu_celeb.h5')
        self.decoder.model = tf.keras.models.load_model('decoder_cae_relu_caleb.h5')

In [6]:
autoencoder = Autoncoder()

In [7]:
autoencoder.load()

In [9]:
test_set_list = tf.data.Dataset.list_files(dir_with_images)

In [10]:
import pandas as pd

df2 = pd.read_csv(dir_with_labels+'labels.csv')

In [12]:
df2.describe()

,faceAttributes_accessories_0_confidence,faceAttributes_accessories_1_confidence,faceAttributes_age,faceAttributes_blur_value,faceAttributes_emotion_anger,faceAttributes_emotion_contempt,faceAttributes_emotion_disgust,faceAttributes_emotion_fear,faceAttributes_emotion_happiness,faceAttributes_emotion_neutral,...,faceAttributes_hair_hairColor_5_confidence,faceAttributes_headPose_pitch,faceAttributes_headPose_roll,faceAttributes_headPose_yaw,faceAttributes_noise_value,faceAttributes_smile,faceRectangle_height,faceRectangle_left,faceRectangle_top,faceRectangle_width
count,4974.000000,256.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,...,55590.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000,58943.000000
mean,0.981659,0.996563,32.970141,0.116848,0.007831,0.004911,0.001372,0.000855,0.540043,0.424203,...,0.039085,-4.782600,0.194498,-1.754719,0.161280,0.540043,88.559575,44.161546,87.155031,88.559541
std,0.040163,0.008111,10.125092,0.196269,0.064451,0.034429,0.019920,0.014048,0.460979,0.445798,...,0.036567,8.202621,2.263451,16.019280,0.220911,0.460979,3.460536,2.210465,1.667967,3.460502
min,0.520000,0.940000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-46.500000,-20.500000,-92.500000,0.000000,0.000000,50.000000,0.000000,71.000000,50.000000
25%,0.980000,1.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002000,0.000000,...,0.010000,-10.200000,-1.000000,-10.700000,0.000000,0.002000,86.000000,43.000000,86.000000,86.000000
50%,0.990000,1.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.723000,0.174000,...,0.030000,-5.200000,0.200000,-1.600000,0.060000,0.723000,88.000000,44.000000,87.000000,88.000000
75%,1.000000,1.000000,36.000000,0.160000,0.000000,0.001000,0.000000,0.000000,1.000000,0.970000,...,0.050000,0.100000,1.400000,6.800000,0.260000,1.000000,90.000000,46.000000,88.000000,90.000000
max,1.000000,1.000000,72.000000,1.000000,1.000000,1.000000,0.934000,0.824000,1.000000,1.000000,...,0.440000,39.300000,29.900000,94.700000,1.000000,1.000000,130.000000,90.000000,103.000000,130.000000


In [13]:
root4 = "D:\\img_align_celeba"

In [14]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [208, 176])

def process_path(file_path):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

In [15]:
from IPython.display import clear_output

In [16]:
after_autoncoder = []
frame = np.empty((1, 208, 176, 3))
for i, image in df2.iterrows(): 
    imId = image['imgId']
    src = os.path.join(root4, imId)
    x = process_path(src)
    frame[0] = x
    z = autoencoder.encode(frame); 
    res = image.values
    res = np.concatenate((np.array(res).flatten(),z.numpy().flatten()), axis=None)
    after_autoncoder.append(res)
    clear_output(wait=False)
    if i % 100:
        print(i)
    

58942


In [19]:
df = pd.DataFrame(after_autoncoder) 

In [20]:
additional = df.columns.values.shape[0] - df2.columns.values.shape[0]

In [21]:
new_cols = np.concatenate((df2.columns.values, [ str(i) for i in range(0,additional)]), axis=None)

In [22]:
import pandas as pd 
df = pd.DataFrame(after_autoncoder, columns=new_cols) 

In [23]:
df.head()

,faceAttributes_accessories_0_confidence,faceAttributes_accessories_0_type,faceAttributes_accessories_1_confidence,faceAttributes_accessories_1_type,faceAttributes_age,faceAttributes_blur_blurLevel,faceAttributes_blur_value,faceAttributes_emotion_anger,faceAttributes_emotion_contempt,faceAttributes_emotion_disgust,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,NaN,NaN,NaN,NaN,33.0,low,0.00,0.0,0.0,0.0,...,-4.909427,5.144699,1.212814,5.816773,2.391655,1.682682,-1.160049,-3.525571,5.361853,-4.666918
1,NaN,NaN,NaN,NaN,39.0,low,0.14,0.0,0.0,0.0,...,-0.162074,1.983305,4.307494,3.899489,1.089170,1.632857,3.319530,-0.384529,-2.099608,0.452602
2,NaN,NaN,NaN,NaN,30.0,medium,0.29,0.0,0.0,0.0,...,-2.732981,3.459917,1.911363,-1.569724,-7.090312,0.352208,0.535671,0.148428,-1.151932,-1.328662
3,NaN,NaN,NaN,NaN,25.0,low,0.03,0.0,0.0,0.0,...,1.626799,-0.813143,4.035781,1.266317,0.950657,1.988803,1.581747,-0.674750,-1.351871,-3.275784
4,NaN,NaN,NaN,NaN,31.0,low,0.00,0.0,0.0,0.0,...,1.352251,-0.621132,-0.513855,2.152973,-0.944837,0.979749,0.330939,0.573130,-2.974519,-0.452726


In [24]:
src = os.path.join(dir_with_labels, "labels_and_encoded_images.csv")
df.to_csv(src,index=False)